In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv
/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip


In [2]:
!pip install dagshub
!pip install mlflow
import dagshub
import mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 5.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: dacite
    Found existing installation: dacite 1.9.2
    Uninstalling dacite-1.9.2:
      Successfully uninstalled dacite-1.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires dacite>=1.8, but you have dacite 1.6.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 65.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.1 MB/s eta 0:00:00
   ━━

In [3]:
!pip install dagshub mlflow
import dagshub
import mlflow
dagshub.init(repo_owner='TamariToradze', repo_name='ML-Final', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=26e7421e-06f1-4040-90b8-de5735fc06a7&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=c8b6fc65f9b22463015e6a9113ad38ed1374a6c2be9419790ec5a58b70bdfa54




Output()

Accessing as TamariToradze

Initialized MLflow to track repo "TamariToradze/ML-Final"

Repository TamariToradze/ML-Final initialized!

# Data Preprocessing

In [4]:
features = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip")
train = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip")
stores = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv")
test = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip")

In [5]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_error
import mlflow
import dagshub
import warnings
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
warnings.filterwarnings('ignore')

In [6]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin


class Merger(BaseEstimator, TransformerMixin):
  
    def __init__(self, auxiliary_features, metadata_info):
       
        # Combine auxiliary data sources into unified reference dataset
        self.reference_data = auxiliary_features.merge(
            metadata_info, 
            how='inner', 
            on='Store'
        )
        # Ensure consistent datetime handling
        self.reference_data['Date'] = pd.to_datetime(self.reference_data['Date'])
    
    def fit(self, X, y=None):
        """
        Fit method required by sklearn interface.
        No actual fitting needed for this transformer.
        """
        return self
    
    def transform(self, X):
    
        # Create working copy to avoid modifying original data
        enhanced_data = X.copy()
        enhanced_data['Date'] = pd.to_datetime(enhanced_data['Date'])
        
        # Join with reference data using common keys
        enriched_dataset = enhanced_data.merge(
            self.reference_data,
            how='inner',
            on=['Store', 'Date', 'IsHoliday']
        )
        
        # Apply consistent ordering for reproducible results
        final_dataset = enriched_dataset.sort_values(
            by=['Date', 'Store', 'Dept']
        ).reset_index(drop=True)
        
        return final_dataset

In [7]:
class DateTimeFeatureExtractor(BaseEstimator, TransformerMixin):
    """Extracts basic datetime components and converts temperature units."""
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        data = X.copy()
        
        # Temperature conversion to Celsius
        if 'Temperature' in data.columns:
            data['Temperature'] = (data['Temperature'] - 32) * (5.0 / 9.0)
        
        # Date component extraction
        data['Day'] = data['Date'].dt.day
        data['Month'] = data['Date'].dt.month
        data['Year'] = data['Date'].dt.year
        
        return data

In [8]:
class HolidayFeatureGenerator(BaseEstimator, TransformerMixin):
    """Generates holiday-related features and proximity indicators."""
    
    def __init__(self):
        self.superbowl_dates = pd.to_datetime(['2010-02-12', '2011-02-11', '2012-02-10', '2013-02-08'])
        self.laborday_dates = pd.to_datetime(['2010-09-10', '2011-09-09', '2012-09-07', '2013-09-06'])
        self.thanksgiving_dates = pd.to_datetime(['2010-11-26', '2011-11-25', '2012-11-23', '2013-11-29'])
        self.christmas_dates = pd.to_datetime(['2010-12-31', '2011-12-30', '2012-12-28', '2013-12-27'])
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        data = X.copy()
        
        # ISO calendar components for holiday detection
        data['WeekNum'] = data['Date'].dt.isocalendar().week
        data['CalendarYear'] = data['Date'].dt.year
        
        # Holiday week detection function
        def check_holiday_week(dates, holiday_list):
            holiday_week_set = set((d.isocalendar().week, d.year) for d in holiday_list)
            return dates.apply(lambda d: (d.isocalendar().week, d.year) in holiday_week_set if pd.notnull(d) else False).astype(int)
        
        data['SuperbowlWeek'] = check_holiday_week(data['Date'], self.superbowl_dates)
        data['LaborDayWeek'] = check_holiday_week(data['Date'], self.laborday_dates)
        data['ThanksgivingWeek'] = check_holiday_week(data['Date'], self.thanksgiving_dates)
        data['ChristmasWeek'] = check_holiday_week(data['Date'], self.christmas_dates)
        
        # Holiday proximity calculations using fixed anchor dates
        thanksgiving_anchor = pd.to_datetime(data['Year'].astype(str) + "-11-24")
        christmas_anchor = pd.to_datetime(data['Year'].astype(str) + "-12-24")
        
        data['Days_to_Thanksgiving'] = (thanksgiving_anchor - data['Date']).dt.days
        data['Days_to_Christmas'] = (christmas_anchor - data['Date']).dt.days
        
        # Remove temporary columns
        data = data.drop(columns=['WeekNum', 'CalendarYear'])
        
        return data

In [9]:
class TemporalFeatureEngineer(BaseEstimator, TransformerMixin):
    """Combined transformer that applies both datetime and holiday feature extraction."""
    
    def __init__(self):
        self.datetime_extractor = DateTimeFeatureExtractor()
        self.holiday_generator = HolidayFeatureGenerator()
    
    def fit(self, X, y=None):
        self.datetime_extractor.fit(X, y)
        self.holiday_generator.fit(X, y)
        return self
    
    def transform(self, X):
        # Apply datetime features first
        data = self.datetime_extractor.transform(X)
        # Then apply holiday features
        data = self.holiday_generator.transform(data)
        return data

In [10]:
class NaFiller(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.promotional_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
        self.economic_cols = ['CPI', 'Unemployment']
        self.computed_means = {}
    
    def fit(self, X, y=None):
        for column in self.economic_cols:
            if column in X.columns:
                self.computed_means[column] = X[column].mean()
        return self
    
    def transform(self, X):
        data = X.copy()
        
        # Fill promotional markdowns with 0
        for column in self.promotional_cols:
            if column in data.columns:
                data[column] = data[column].fillna(0.0)
        
        # Fill economic indicators with computed mean
        for column in self.economic_cols:
            if column in data.columns and column in self.computed_means:
                data[column] = data[column].fillna(self.computed_means[column])
        
        return data

In [11]:
class CategoryMapper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.store_type_map = {'A': 3, 'B': 2, 'C': 1}
        self.boolean_flag_map = {False: 0, True: 1}
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        data = X.copy()
        
        if 'Type' in data.columns:
            data['Type'] = data['Type'].map(self.store_type_map)
        
        if 'IsHoliday' in data.columns:
            data['IsHoliday'] = data['IsHoliday'].map(self.boolean_flag_map)
        
        return data

In [12]:
class StoreDataProcessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.processed_data = {}
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        self.processed_data = {}
        for store_id in X['Store'].unique():
            self.process_store_data(store_id, X)
        return self.processed_data
    
    def process_store_data(self, store_num, X):
        store_subset = X[X['Store'] == store_num].copy()
        
        # Detect if sales data is available (training vs test scenario)
        sales_available = 'Weekly_Sales' in store_subset.columns
        
        if sales_available:
            sum_fields = ['Weekly_Sales', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
        else:
            sum_fields = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
        
        first_fields = ['IsHoliday', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
                       'Type', 'Size', 'Day', 'Month', 'Year', 'SuperbowlWeek',
                       'LaborDayWeek', 'ThanksgivingWeek', 'ChristmasWeek',
                       'Days_to_Thanksgiving', 'Days_to_Christmas']
        
        aggregation_rules = {}
        
        # Add summation rules for existing columns
        for field in sum_fields:
            if field in store_subset.columns:
                aggregation_rules[field] = 'sum'
        
        # Add first-value rules for existing columns
        for field in first_fields:
            if field in store_subset.columns:
                aggregation_rules[field] = 'first'
        
        consolidated = store_subset.groupby(['Date', 'Store']).agg(aggregation_rules).reset_index()
        consolidated = consolidated.sort_values('Date').reset_index(drop=True)
        
        # Compute department proportions only when sales data exists
        if sales_available:
            dept_ratios = self.compute_dept_ratios(store_subset)
        else:
            dept_ratios = None
        
        self.processed_data[store_num] = (consolidated, dept_ratios)
        return consolidated
    
    def compute_dept_ratios(self, store_subset):
        dept_sales_totals = store_subset.groupby('Dept')['Weekly_Sales'].sum()
        overall_total = store_subset['Weekly_Sales'].sum()
        
        if overall_total == 0:
            dept_count = len(dept_sales_totals)
            return {dept: 1.0/dept_count for dept in dept_sales_totals.index}
        
        dept_ratios_dict = (dept_sales_totals / overall_total).to_dict()
        return dept_ratios_dict

In [13]:
from sklearn.pipeline import Pipeline


In [14]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('merge', Merger(features, stores)),
    ('DateTimeFeatureExtractor', DateTimeFeatureExtractor()),
    ('HolidayFeatureGenerator', HolidayFeatureGenerator()),
    ('TemporalFeatureEngineer',TemporalFeatureEngineer()),
    ('value_fill', NaFiller()),
    ('cat_encoder', CategoryMapper()),
    ('StoreDataProcessor', StoreDataProcessor())
])

In [15]:
train_dict = pipeline.fit_transform(train)
test_dict = pipeline.transform(test)


# Training

In [16]:
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [17]:
!pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.6/200.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 62.1 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.0/340.0 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:0

In [18]:
!pip install scikit-learn==1.3.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 73.9 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.7.1
    Uninstalling scikit-learn-1.7.1:
      Successfully uninstalled scikit-learn-1.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
darts 0.36.0 requires scikit-learn>=1.6.0, but you have scikit-learn 1.3.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


In [19]:
import warnings
warnings.filterwarnings('ignore')
from prophet import Prophet
from tqdm import tqdm
import pandas as pd
import numpy as np

# Define external variables for forecasting
regressor_features = ['IsHoliday', 'SuperbowlWeek', 'LaborDayWeek', 'ThanksgivingWeek', 'ChristmasWeek']

# Dictionary to store trained models
trained_models = {}

# Train Prophet models for each store
for store in tqdm(train_dict.keys(), desc="Building Prophet forecasting models"):
   time_series_data, department_proportions = train_dict[store]
   
   try:
       # Format data for Prophet requirements
       prophet_df = time_series_data[['Date', 'Weekly_Sales']].copy()
       prophet_df.rename(columns={'Date': 'ds', 'Weekly_Sales': 'y'}, inplace=True)
       prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])
       
       # Identify which regressors are present in the dataset
       valid_regressors = []
       for feature in regressor_features:
           if feature in time_series_data.columns:
               prophet_df[feature] = time_series_data[feature].values
               valid_regressors.append(feature)
       
       # Organize data chronologically
       prophet_df = prophet_df.sort_values('ds').reset_index(drop=True)
       
       # Configure Prophet forecasting model
       forecaster = Prophet(
           yearly_seasonality=True,
           weekly_seasonality=False,
           daily_seasonality=False,
           changepoint_prior_scale=0.01,
           seasonality_prior_scale=10.0,
           holidays_prior_scale=15.0,
           mcmc_samples=0,
           interval_width=0.8,
           growth='flat'
       )
       
       # Include US holiday effects
       forecaster.add_country_holidays(country_name='US')
       
       # Register external regressors
       for feature in valid_regressors:
           forecaster.add_regressor(feature)
       
       # Train the forecasting model
       forecaster.fit(prophet_df)
       
       # Save trained model with metadata
       trained_models[store] = (forecaster, department_proportions, valid_regressors)
       
   except Exception as error:
       print(f"Error training Prophet model for Store {store}: {error}")
       continue

print(f"Completed training for {len(trained_models)} store forecasting models")

Building Prophet forecasting models:   0%|          | 0/45 [00:00<?, ?it/s]20:04:01 - cmdstanpy - INFO - Chain [1] start processing
20:04:01 - cmdstanpy - INFO - Chain [1] done processing
Building Prophet forecasting models:   2%|▏         | 1/45 [00:01<01:10,  1.60s/it]20:04:01 - cmdstanpy - INFO - Chain [1] start processing
20:04:01 - cmdstanpy - INFO - Chain [1] done processing
20:04:01 - cmdstanpy - INFO - Chain [1] start processing
20:04:01 - cmdstanpy - INFO - Chain [1] done processing
Building Prophet forecasting models:   7%|▋         | 3/45 [00:01<00:20,  2.10it/s]20:04:01 - cmdstanpy - INFO - Chain [1] start processing
20:04:01 - cmdstanpy - INFO - Chain [1] done processing
20:04:01 - cmdstanpy - INFO - Chain [1] start processing
20:04:01 - cmdstanpy - INFO - Chain [1] done processing
Building Prophet forecasting models:  11%|█         | 5/45 [00:01<00:10,  3.64it/s]20:04:01 - cmdstanpy - INFO - Chain [1] start processing
20:04:01 - cmdstanpy - INFO - Chain [1] done processin

Completed training for 45 store forecasting models


# Predictions

In [28]:
import mlflow
import dagshub
import pickle
import os
from datetime import datetime

# Initialize MLflow experiment tracking
mlflow.set_experiment("Prophet_Forecasting")

# Begin MLflow tracking session
with mlflow.start_run(run_name=f"Prophet_Training_Run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
    
    # Record experiment configuration
    mlflow.log_param("forecasting_model", "Prophet")
    mlflow.log_param("methodology", "store_aggregated_approach")
    mlflow.log_param("external_features", ", ".join(regressor_features))
    mlflow.log_param("feature_count", len(regressor_features))
    mlflow.log_param("training_stores", len(train_dict))
    mlflow.log_param("trained_models", len(trained_models))
    mlflow.log_param("training_failures", len(train_dict) - len(trained_models))
    
    # Record Prophet configuration parameters
    mlflow.log_param("annual_patterns", True)
    mlflow.log_param("weekly_patterns", False)
    mlflow.log_param("daily_patterns", False)
    mlflow.log_param("trend_flexibility", 0.01)
    mlflow.log_param("seasonal_strength", 10.0)
    mlflow.log_param("holiday_strength", 15.0)
    mlflow.log_param("sampling_method", 0)
    mlflow.log_param("confidence_interval", 0.8)
    mlflow.log_param("trend_type", "flat")
    mlflow.log_param("holiday_calendar", "US")
    
    # Calculate and log performance metrics
    success_ratio = len(trained_models) / len(train_dict)
    mlflow.log_metric("training_success_ratio", success_ratio)
    
    # Analyze store-level information and calculate performance metrics
    store_statistics = []
    total_time_periods = 0
    total_dept_count = 0
    all_mape_scores = []
    all_rmse_scores = []
    
    for store_key, (model, dept_data, features) in trained_models.items():
        # Extract training data size
        training_data = train_dict[store_key][0]
        training_size = len(training_data)
        dept_size = len(dept_data) if dept_data is not None else 0
        
        total_time_periods += training_size
        total_dept_count += dept_size
        
        # Prepare data for evaluation
        eval_df = training_data[['Date', 'Weekly_Sales']].copy()
        eval_df.columns = ['ds', 'y']
        eval_df['ds'] = pd.to_datetime(eval_df['ds'])
        
        # Add regressors for prediction
        for feature in features:
            if feature in training_data.columns:
                eval_df[feature] = training_data[feature].values
        
        # Generate predictions on training data
        try:
            predictions = model.predict(eval_df)
            actual_values = eval_df['y'].values
            predicted_values = predictions['yhat'].values
            
            # Calculate MAPE (Mean Absolute Percentage Error)
            mape = np.mean(np.abs((actual_values - predicted_values) / actual_values)) * 100
            
            # Calculate RMSE (Root Mean Square Error)
            rmse = np.sqrt(np.mean((actual_values - predicted_values) ** 2))
            
            all_mape_scores.append(mape)
            all_rmse_scores.append(rmse)
            
        except Exception as e:
            print(f"Error calculating metrics for Store {store_key}: {e}")
            mape, rmse = None, None
        
        store_info = {
            'store_id': store_key,
            'training_periods': training_size,
            'department_count': dept_size,
            'active_features': len(features),
            'mape': mape,
            'rmse': rmse
        }
        store_statistics.append(store_info)
    
    # Calculate and record aggregate statistics
    if len(trained_models) > 0:
        avg_periods = total_time_periods / len(trained_models)
        avg_departments = total_dept_count / len(trained_models)
        
        # Calculate performance metrics
        valid_mape_scores = [score for score in all_mape_scores if score is not None]
        valid_rmse_scores = [score for score in all_rmse_scores if score is not None]
        
        if valid_mape_scores:
            avg_mape = np.mean(valid_mape_scores)
            median_mape = np.median(valid_mape_scores)
            mlflow.log_metric("average_mape", avg_mape)
            mlflow.log_metric("median_mape", median_mape)
            mlflow.log_metric("best_mape", min(valid_mape_scores))
            mlflow.log_metric("worst_mape", max(valid_mape_scores))
        
        if valid_rmse_scores:
            avg_rmse = np.mean(valid_rmse_scores)
            median_rmse = np.median(valid_rmse_scores)
            mlflow.log_metric("average_rmse", avg_rmse)
            mlflow.log_metric("median_rmse", median_rmse)
            mlflow.log_metric("best_rmse", min(valid_rmse_scores))
            mlflow.log_metric("worst_rmse", max(valid_rmse_scores))
        
        mlflow.log_metric("average_training_periods", avg_periods)
        mlflow.log_metric("average_departments", avg_departments)
        mlflow.log_metric("combined_departments", total_dept_count)
        
        # Print comprehensive metrics summary
        print("=" * 60)
        print("PROPHET MODEL TRAINING SUMMARY")
        print("=" * 60)
        print(f"Experiment Name: Prophet_Forecasting")
        print(f"Training Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print()
        
        print("TRAINING OVERVIEW:")
        print(f"  • Total Stores in Dataset: {len(train_dict)}")
        print(f"  • Successfully Trained Models: {len(trained_models)}")
        print(f"  • Failed Training Attempts: {len(train_dict) - len(trained_models)}")
        print(f"  • Training Success Rate: {success_ratio:.1%}")
        print()
        
        print("MODEL CONFIGURATION:")
        print(f"  • Forecasting Algorithm: Prophet")
        print(f"  • External Features Used: {', '.join(regressor_features)}")
        print(f"  • Feature Count: {len(regressor_features)}")
        print(f"  • Trend Type: Flat")
        print(f"  • Holiday Calendar: US")
        print(f"  • Confidence Interval: 80%")
        print()
        
        print("DATA STATISTICS:")
        print(f"  • Average Training Periods per Store: {avg_periods:.1f}")
        print(f"  • Average Departments per Store: {avg_departments:.1f}")
        print(f"  • Total Departments Across All Stores: {total_dept_count}")
        print(f"  • Total Training Data Points: {total_time_periods}")
        print()
        
        # Print performance metrics
        if valid_mape_scores:
            print("PERFORMANCE METRICS (MAPE - Mean Absolute Percentage Error):")
            print(f"  • Average MAPE: {avg_mape:.2f}%")
            print(f"  • Median MAPE: {median_mape:.2f}%")
            print(f"  • Best MAPE: {min(valid_mape_scores):.2f}%")
            print(f"  • Worst MAPE: {max(valid_mape_scores):.2f}%")
            print()
        
        print("TOP 5 STORES BY PERFORMANCE (Lowest MAPE):")
        valid_stores = [s for s in store_statistics if s['mape'] is not None]
        if valid_stores:
            sorted_by_mape = sorted(valid_stores, key=lambda x: x['mape'])[:5]
            for i, store in enumerate(sorted_by_mape, 1):
                print(f"  {i}. Store {store['store_id']}: MAPE={store['mape']:.2f}%, "
                      f"{store['training_periods']} periods")
        print()
        
        print("TOP 5 STORES BY TRAINING DATA SIZE:")
        sorted_stores = sorted(store_statistics, key=lambda x: x['training_periods'], reverse=True)[:5]
        for i, store in enumerate(sorted_stores, 1):
            mape_str = f"MAPE={store['mape']:.2f}%" if store['mape'] is not None else "MAPE=N/A"
            print(f"  {i}. Store {store['store_id']}: {store['training_periods']} periods, "
                  f"{store['department_count']} depts, {mape_str}")
        
        print("=" * 60)
    
    else:
        print("WARNING: No models were successfully trained!")
        mlflow.log_metric("average_training_periods", 0)
        mlflow.log_metric("average_departments", 0)
        mlflow.log_metric("combined_departments", 0)

PROPHET MODEL TRAINING SUMMARY
Experiment Name: Prophet_Forecasting
Training Timestamp: 2025-07-29 20:45:58

TRAINING OVERVIEW:
  • Total Stores in Dataset: 45
  • Successfully Trained Models: 45
  • Failed Training Attempts: 0
  • Training Success Rate: 100.0%

MODEL CONFIGURATION:
  • Forecasting Algorithm: Prophet
  • External Features Used: IsHoliday, SuperbowlWeek, LaborDayWeek, ThanksgivingWeek, ChristmasWeek
  • Feature Count: 5
  • Trend Type: Flat
  • Holiday Calendar: US
  • Confidence Interval: 80%

DATA STATISTICS:
  • Average Training Periods per Store: 143.0
  • Average Departments per Store: 74.0
  • Total Departments Across All Stores: 3331
  • Total Training Data Points: 6435

PERFORMANCE METRICS (MAPE - Mean Absolute Percentage Error):
  • Average MAPE: 5.02%
  • Median MAPE: 4.50%
  • Best MAPE: 2.13%
  • Worst MAPE: 13.53%

TOP 5 STORES BY PERFORMANCE (Lowest MAPE):
  1. Store 37: MAPE=2.13%, 143 periods
  2. Store 30: MAPE=3.08%, 143 periods
  3. Store 10: MAPE=3.1

In [21]:
max_training_date = pd.to_datetime(train['Date'].max())

def generate_prediction(sample_idx):
    """Generate forecast for a specific test sample"""
    test_sample = test.iloc[sample_idx]
    store_id = test_sample['Store']
    department_id = test_sample['Dept']
    forecast_date = pd.to_datetime(test_sample['Date'])
    
    try:
        # Retrieve trained model and metadata
        forecasting_model, department_weights, valid_features = trained_models[store_id]
        
        # Verify store exists in test dataset
        if store_id not in test_dict:
            return 1000.0
            
        # Load and prepare test data for the store
        store_test_data, *_ = test_dict[store_id]
        store_test_data = store_test_data.copy()
        store_test_data['Date'] = pd.to_datetime(store_test_data['Date'])
        store_test_data = store_test_data.sort_values('Date')
        
        # Filter data up to prediction date
        filtered_data = store_test_data[store_test_data['Date'] <= forecast_date].copy()
        if len(filtered_data) == 0:
            return 1000.0
        
        # Construct input dataframe for Prophet prediction
        prediction_input = {
            'ds': filtered_data['Date'].tolist()
        }
        
        # Include external regressors that are available
        for feature_name in valid_features:
            if feature_name in filtered_data.columns:
                prediction_input[feature_name] = filtered_data[feature_name].tolist()
        
        input_df = pd.DataFrame(prediction_input)
        
        # Generate Prophet forecast
        forecast_result = forecasting_model.predict(input_df)
        forecast_result['ds'] = pd.to_datetime(forecast_result['ds'])
        
        # Extract prediction for target date
        date_specific_forecast = forecast_result[forecast_result['ds'] == forecast_date]
        
        if len(date_specific_forecast) > 0:
            base_prediction = max(0, date_specific_forecast['yhat'].iloc[0])
            
            # Apply department-specific weighting
            if department_weights and department_id in department_weights.keys():
                return base_prediction * department_weights[department_id]
            elif department_weights:
                return base_prediction / len(department_weights)
            else:
                return base_prediction
        else:
            return 1000.0  # Default value when date not found
            
    except Exception as error:
        print(f"Prediction failed for Store {store_id}, Department {department_id}: {error}")
        return 1000.0  # Error fallback value

In [22]:
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

def handle_store_predictions(store_info_tuple):
    """Generate all forecasts for a single store in batch mode"""
    store_identifier, store_samples = store_info_tuple

    # Verify model availability for this store
    if store_identifier not in trained_models:
        # Generate fallback results for all samples
        prediction_results = []
        for _, sample_row in store_samples.iterrows():
            prediction_results.append({
                'Id': f"{sample_row['Store']}_{sample_row['Dept']}_{pd.to_datetime(sample_row['Date']).strftime('%Y-%m-%d')}",
                'Weekly_Sales': 1000.0
            })
        return prediction_results

    try:
        forecasting_model, department_ratios, model_features = trained_models[store_identifier]

        # Retrieve aggregated test data for store
        if store_identifier not in test_dict:
            # Handle missing aggregated data
            prediction_results = []
            for _, sample_row in store_samples.iterrows():
                prediction_results.append({
                    'Id': f"{sample_row['Store']}_{sample_row['Dept']}_{pd.to_datetime(sample_row['Date']).strftime('%Y-%m-%d')}",
                    'Weekly_Sales': 1000.0
                })
            return prediction_results

        aggregated_data, _ = test_dict[store_identifier]
        aggregated_data = aggregated_data.copy()
        aggregated_data['Date'] = pd.to_datetime(aggregated_data['Date'])
        aggregated_data = aggregated_data.sort_values('Date')

        # Extract all required prediction dates
        store_samples['Date'] = pd.to_datetime(store_samples['Date'])
        prediction_dates = sorted(store_samples['Date'].unique())

        # Build comprehensive input for Prophet
        prophet_input = {'ds': prediction_dates}

        # Map external features to all dates
        for feature_col in model_features:
            if feature_col in aggregated_data.columns:
                # Extract feature values for each prediction date
                feature_mapping = []
                for target_date in prediction_dates:
                    # Get most recent feature value up to target date
                    historical_data = aggregated_data[aggregated_data['Date'] <= target_date]
                    if len(historical_data) > 0:
                        feature_mapping.append(historical_data[feature_col].iloc[-1])
                    else:
                        feature_mapping.append(0)  # Default value

                prophet_input[feature_col] = feature_mapping

        prophet_df = pd.DataFrame(prophet_input)

        # Execute single batch prediction for all dates
        batch_forecast = forecasting_model.predict(prophet_df)
        batch_forecast['ds'] = pd.to_datetime(batch_forecast['ds'])

        # Build lookup table for date-based predictions
        prediction_lookup = dict(zip(batch_forecast['ds'], batch_forecast['yhat']))

        # Generate final predictions for all test samples
        prediction_results = []
        for _, sample_row in store_samples.iterrows():
            target_date = pd.to_datetime(sample_row['Date'])
            department = sample_row['Dept']

            # Retrieve store-level forecast
            if target_date in prediction_lookup:
                base_forecast = max(0, prediction_lookup[target_date])

                # Calculate department-specific allocation
                if department_ratios and department in department_ratios:
                    adjusted_prediction = base_forecast * department_ratios[department]
                elif department_ratios:
                    adjusted_prediction = base_forecast / len(department_ratios)
                else:
                    adjusted_prediction = base_forecast
            else:
                adjusted_prediction = 1000.0

            prediction_results.append({
                'Id': f"{sample_row['Store']}_{sample_row['Dept']}_{target_date.strftime('%Y-%m-%d')}",
                'Weekly_Sales': max(0, adjusted_prediction)
            })

        return prediction_results

    except Exception as error:
        print(f"Batch forecasting failed for Store {store_identifier}: {error}")
        prediction_results = []
        for _, sample_row in store_samples.iterrows():
            prediction_results.append({
                'Id': f"{sample_row['Store']}_{sample_row['Dept']}_{pd.to_datetime(sample_row['Date']).strftime('%Y-%m-%d')}",
                'Weekly_Sales': 1000.0
            })
        return prediction_results

# Group test data by store for parallel processing
stores_grouped = list(test.groupby('Store'))

# Execute parallel batch processing
final_submission = []
with ThreadPoolExecutor(max_workers=4) as thread_pool:
    prediction_tasks = [thread_pool.submit(handle_store_predictions, store_group) for store_group in stores_grouped]

    for completed_task in tqdm(as_completed(prediction_tasks), total=len(prediction_tasks)):
        store_predictions = completed_task.result()
        final_submission.extend(store_predictions)

100%|██████████| 45/45 [00:19<00:00,  2.27it/s]


In [27]:
# Use the existing batch processing function but simplify output
stores_grouped = list(test.groupby('Store'))
all_predictions = []

print(f"Processing {len(stores_grouped)} stores in parallel...")

with ThreadPoolExecutor(max_workers=4) as executor:
    futures = [executor.submit(handle_store_predictions, store_group) for store_group in stores_grouped]
    
    for future in tqdm(as_completed(futures), total=len(futures)):
        store_results = future.result()
        all_predictions.extend(store_results)

# Convert to DataFrame and save
forecast_df = pd.DataFrame(all_predictions)
forecast_df = forecast_df.sort_values('Id').reset_index(drop=True)
forecast_df.to_csv('prophet_predictions.csv', index=False)

print(f"Saved {len(forecast_df)} predictions to prophet_predictions.csv")
print(f"Average prediction: ${forecast_df['Weekly_Sales'].mean():.2f}")
print(f"Fallback predictions (1000.0): {len(forecast_df[forecast_df['Weekly_Sales'] == 1000.0])}")

Processing 45 stores in parallel...


100%|██████████| 45/45 [00:19<00:00,  2.29it/s]


Saved 115064 predictions to prophet_predictions.csv
Average prediction: $16230.79
Fallback predictions (1000.0): 0
